# Testing the SinglePlaneLOS() class

This notebook aims to test the features of the line-of-sight branch of lenstronomy, which includes the SinglePlaneLOS() class and the two line-of-sight models: 'LOS' and 'LOS_MINIMAL'. In particular, we check that a lens with line-of-sight corrections is equivalent to a multi-plane lens setting.

## 1. Setting up the lens models

In [24]:
import numpy as np

### Shears Model

Define a lens with line-of-sight shear using the standard approach of lenstronomy, i.e. add 3 shear planes to the main lens, here modelled as an elliptical power law (EPL).

In [25]:
lens_model_list_shears = ['EPL', 'SHEAR', 'SHEAR', 'SHEAR']

# Parameters of the main lens
# We define its centre as the line of sight
kwargs_epl = {'theta_E': 0.8,
               'gamma': 1.95,
               'center_x': 0,
               'center_y': 0,
               'e1': .07,
               'e2': -0.03}

# LOS (linear) shears
gamma_od = np.array([0.05, -0.1])
gamma_os = np.array([0.02, 0.1])
gamma_ds = np.array([0., 0.03])

# Cosmology (we need it for the distances that are involved in the relations between shears above)
from astropy.cosmology import default_cosmology
cosmo = default_cosmology.get()

# Lensing setup to generate the 'true' (mock) image:
# a main lens (EPL) and three shear planes
z_s = 2
z_d = 0.5 # redshift of dominant lens
z_f = z_d/2 # redshift of foreground shear plane
z_b = (z_s + z_d)/2 # redshift of background shear plane
lens_redshift_list_shear = [z_d, z_f, z_d, z_b]

# Convert line-of-sight shears into lenstronomy shears
gamma_f = gamma_od*(
    cosmo.angular_diameter_distance_z1z2(0, z_d)
    * cosmo.angular_diameter_distance_z1z2(z_f, z_s)
    / cosmo.angular_diameter_distance_z1z2(0, z_s)
    / cosmo.angular_diameter_distance_z1z2(z_f, z_d))
gamma_b = gamma_ds*(
    cosmo.angular_diameter_distance_z1z2(0, z_b)
    * cosmo.angular_diameter_distance_z1z2(z_d, z_s)
    / cosmo.angular_diameter_distance_z1z2(0, z_s)
    / cosmo.angular_diameter_distance_z1z2(z_d, z_b))
gamma_d = gamma_os - gamma_f - gamma_b
           
kwargs_gamma_f = {'gamma1': float(gamma_f[0]), 'gamma2': float(gamma_f[1])}
kwargs_gamma_d = {'gamma1': float(gamma_d[0]), 'gamma2': float(gamma_d[1])}
kwargs_gamma_b = {'gamma1': float(gamma_b[0]), 'gamma2': float(gamma_b[1])}
# I used float() here because otherwise astropy complains about units

# Generate the complete lens model
kwargs_lens_shears = [kwargs_epl,
                      kwargs_gamma_f,
                      kwargs_gamma_d,
                      kwargs_gamma_b]
from lenstronomy.LensModel.lens_model import LensModel
lens_model_class_shears = LensModel(
    lens_model_list_shears,
    lens_redshift_list=lens_redshift_list_shear,
    z_lens=z_d,
    z_source=z_s,
    multi_plane=True)

### LOS Model

Define an EPL with line-of-sight shear corrections.

<div class="alert alert-block alert-info">
    
Importantly, in the EPL + 3 shear planes defined above, there are non-linear couplings between the shear planes which lead effectively to second-order convergence and rotation terms in the line-of-sight corrections. These must be taken into account.
    
Let's call $\boldsymbol{\Gamma}_f, \boldsymbol{\Gamma}_d, \boldsymbol{\Gamma}_b$ the three shear matrices in the foreground, dominant, and background planes. The exact displacement angle in that case reads
$$
\boldsymbol{\alpha}(\boldsymbol{\theta})
=
\boldsymbol{\Gamma}_{os}\boldsymbol{\theta}
+ (1-\boldsymbol{\Gamma}_{ds})
    \boldsymbol{\alpha}_{ods}[(1-\boldsymbol{\Gamma}_{od})\boldsymbol{\theta}]
$$
with
\begin{align}
\boldsymbol{\Gamma}_{od}
&= \boldsymbol{\Gamma}_{ofd}
= \frac{D_{fd}D_{os}}{D_{od}D_{fs}} \boldsymbol{\Gamma}_{f}
\\
\boldsymbol{\Gamma}_{ds}
&= \boldsymbol{\Gamma}_{dbs}
= \frac{D_{os}D_{db}}{D_{ob}D_{ds}} \boldsymbol{\Gamma}_{b}
\\
\boldsymbol{\Gamma}_{ofb}
&= \frac{D_{os}D_{fb}}{D_{ob}D_{fs}} \boldsymbol{\Gamma}_{f}
\\
\boldsymbol{\Gamma}_{odb}
&= \frac{D_{os}D_{db}}{D_{ob}D_{ds}} \boldsymbol{\Gamma}_{d}
\\
\boldsymbol{\Gamma}_{os}
&= \boldsymbol{\Gamma}_{f} + \boldsymbol{\Gamma}_{d} + \boldsymbol{\Gamma}_{b}
    - \left[
        \boldsymbol{\Gamma}_{d} \boldsymbol{\Gamma}_{od}
        + \boldsymbol{\Gamma}_{b} \boldsymbol{\Gamma}_{ofb}
        + \boldsymbol{\Gamma}_{b} \boldsymbol{\Gamma}_{odb} (1-\boldsymbol{\Gamma}_{od})
        \right]
\end{align}

In [26]:
lens_model_list_los = ['EPL', 'LOS']

# The (od) and (ds) corrections are the ones that we have entered first:
kwargs_los = {
    'kappa_od': 0, 'kappa_ds': 0,
    'omega_od': 0, 'omega_ds': 0,
    'gamma1_od': gamma_od[0], 'gamma2_od': gamma_od[1],
    'gamma1_ds': gamma_ds[0], 'gamma2_ds': gamma_ds[1]}

# but the (os) perturbations are affected by non-linear couplings
Id = np.array([[1, 0],
               [0, 1]])
Gamma_f = np.array([[float(gamma_f[0]), float(gamma_f[1])],
                    [float(gamma_f[1]), -float(gamma_f[0])]])
Gamma_d = np.array([[float(gamma_d[0]), float(gamma_d[1])],
                    [float(gamma_d[1]), -float(gamma_d[0])]])
Gamma_b = np.array([[float(gamma_b[0]), float(gamma_b[1])],
                    [float(gamma_b[1]), -float(gamma_b[0])]])
Gamma_od = np.array([[gamma_od[0], gamma_od[1]],
                     [gamma_od[1], -gamma_od[0]]])
Gamma_ofb = float(cosmo.angular_diameter_distance_z1z2(0, z_s)
                 * cosmo.angular_diameter_distance_z1z2(z_f, z_b)
                 / cosmo.angular_diameter_distance_z1z2(0, z_b)
                 / cosmo.angular_diameter_distance_z1z2(z_f, z_s)) * np.array(Gamma_f)
Gamma_odb = float(cosmo.angular_diameter_distance_z1z2(0, z_s)
                 * cosmo.angular_diameter_distance_z1z2(z_d, z_b)
                 / cosmo.angular_diameter_distance_z1z2(0, z_b)
                 / cosmo.angular_diameter_distance_z1z2(z_d, z_s)) * np.array(Gamma_d)

Gamma_os = (Gamma_f + Gamma_d + Gamma_b
            - np.matmul(Gamma_d, Gamma_od)
            - np.matmul(Gamma_b, Gamma_ofb + np.matmul(Gamma_odb, Id - Gamma_od))
           )

kappa_os = (Gamma_os[0,0] + Gamma_os[1,1]) / 2
gamma1_os = (Gamma_os[0,0] - Gamma_os[1,1]) / 2
gamma2_os = (Gamma_os[0,1] + Gamma_os[1,0]) / 2
omega_os = (Gamma_os[1,0] - Gamma_os[0,1]) / 2

kwargs_los.update({'kappa_os': 0, 'omega_os': 0,
                   'gamma1_os': float(gamma_os[0]), 'gamma2_os': float(gamma_os[1])})

kwargs_los.update({'kappa_os': kappa_os, 'omega_os': omega_os,
                   'gamma1_os': gamma1_os, 'gamma2_os': gamma2_os})

print("""------
For reference, the non-linear convergence and rotation are:
kappa_os = {}
omega_os = {}
while the non-linear corrections to the shear are:
gamma_os - (gamma_f + gamma_d + gamma_b) = {}
------""".format(
    kappa_os, omega_os,
    np.array([gamma1_os, gamma2_os]) - gamma_f - gamma_d - gamma_b))

# Generate the full model
kwargs_lens_los = [kwargs_epl, kwargs_los]
from lenstronomy.LensModel.lens_model import LensModel
lens_model_class_los = LensModel(lens_model_list_los,
    z_lens=z_d,
    z_source=z_s)

------
For reference, the non-linear convergence and rotation are:
kappa_os = 0.02571553336450699
omega_os = -0.006100000000000001
while the non-linear corrections to the shear are:
gamma_os - (gamma_f + gamma_d + gamma_b) = [ 0.00015415 -0.00080625]
------
Adding LOS to the main lens.


## 2. Test of the ray-shooting function

I've kept the ray-shooting and Hessian tests just as confirmation that my lens models are in fact initialised correctly

In [27]:
# List of positions in the image plane
x, y = np.array([3,4,5]), np.array([2,1,0])
x, y = np.array([3]), np.array([2])

# Apply ray shooting to get the associated sources
beta_x_shears, beta_y_shears = lens_model_class_shears.ray_shooting(x, y, kwargs_lens_shears)
beta_x_los, beta_y_los = lens_model_class_los.ray_shooting(x, y, kwargs_lens_los)

print("""
For a list of images with coordinates:
x = {}
y = {}
The sources computed with multi-plane lensing are:
beta_x = {}
beta_y = {}
and the ones from a single plane with LOS corrections are:
beta_x = {}
beta_y = {}

The difference is:
delta beta_x = {}
delta beta_y = {}
""".format(x, y, beta_x_shears, beta_y_shears, beta_x_los, beta_y_los,
           beta_x_shears - beta_x_los, beta_y_shears - beta_y_los
))


For a list of images with coordinates:
x = [3]
y = [2]
The sources computed with multi-plane lensing are:
beta_x = [2.01372567]
beta_y = [1.15719699]
and the ones from a single plane with LOS corrections are:
beta_x = [2.01372567]
beta_y = [1.15719699]

The difference is:
delta beta_x = [-4.4408921e-16]
delta beta_y = [-2.22044605e-16]



The difference is thus machine-precision level.

## 3. Test of the Hessian matrix function

In [28]:
# List of positions in the image plane
x, y = np.array([3,4,5]), np.array([2,1,0])

# Apply the Hessian matrix
H_xx_shears, H_xy_shears, H_yx_shears, H_yy_shears = lens_model_class_shears.hessian(x, y, kwargs_lens_shears)
H_xx_los, H_xy_los, H_yx_los, H_yy_los = lens_model_class_los.hessian(x, y, kwargs_lens_los)

print("""
For a list of images with coordinates:
x = {}
y = {}
The Hessian matrices computed with multi-plane lensing are:
H_xx = {}
H_xy = {}
H_yx = {}
H_yy = {}
and the ones from a single plane with LOS corrections are:
H_xx = {}
H_xy = {}
H_yx = {}
H_yy = {}

The difference is:
delta H_xx = {}
delta H_xy = {}
delta H_yx = {}
delta H_yy = {}
""".format(x, y,
           H_xx_shears, H_xy_shears, H_yx_shears, H_yy_shears,
           H_xx_los, H_xy_los, H_yx_los, H_yy_los,
           H_xx_shears - H_xx_los, H_xy_shears - H_xy_los,
           H_yx_shears - H_yx_los, H_yy_shears - H_yy_los
))


For a list of images with coordinates:
x = [3 4 5]
y = [2 1 0]
The Hessian matrices computed with multi-plane lensing are:
H_xx = [0.12383605 0.07363234 0.05429661]
H_xy = [0.00429612 0.03318208 0.08036052]
H_yx = [0.00902487 0.0481959  0.09400629]
H_yy = [0.14547454 0.20085891 0.20657697]
and the ones from a single plane with LOS corrections are:
H_xx = [0.12383607 0.07363238 0.05429655]
H_xy = [0.00429611 0.0331821  0.08036043]
H_yx = [0.00902492 0.04819587 0.0940063 ]
H_yy = [0.14547461 0.2008589  0.20657696]

The difference is:
delta H_xx = [-1.63872994e-08 -3.39644950e-08  5.69407330e-08]
delta H_xy = [ 5.17911217e-09 -1.58417897e-08  9.10747397e-08]
delta H_yx = [-5.02853696e-08  3.67822338e-08 -4.05713098e-09]
delta H_yy = [-6.63006960e-08  1.19082240e-08  7.69328762e-09]



The difference is slightly larger than in the ray-shooting case, but still very small.

## 4. Test of the Fermat Potential and Time Delays

### Solving the Lens Equation

In [29]:
ra_source, dec_source = 0.05, 0.02

from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver

lensEquationSolver_shears = LensEquationSolver(lens_model_class_shears)
x_image_shears, y_image_shears = lensEquationSolver_shears.findBrightImage(ra_source, dec_source, kwargs_lens_shears, numImages=4)

lensEquationSolver_los = LensEquationSolver(lens_model_class_los)
x_image_los, y_image_los = lensEquationSolver_los.findBrightImage(ra_source, dec_source, kwargs_lens_los, numImages=4)


### Fermat Potential and Time Delays

In [30]:
#Testing time delays

t_days_shears = lens_model_class_shears.arrival_time(x_image_shears, y_image_shears,kwargs_lens_shears)
dt_days_shears =  t_days_shears[1:] - t_days_shears[0]

t_days_los = lens_model_class_los.arrival_time(x_image_los, y_image_los,kwargs_lens_los)
dt_days_los =  t_days_los[1:] - t_days_los[0]

print("""
For a list of images with coordinates:
x = {}
y = {}
The relative time delays computed with multi-plane lensing are:
dt_days = {}
and the ones from a single plane with LOS corrections are:
dt_days = {}

The difference is:
delta dt_days = {}
""".format(x_image_shears, y_image_shears, dt_days_shears, dt_days_los,
           dt_days_shears - dt_days_los
))


For a list of images with coordinates:
x = [ 0.62566472 -0.41057627  0.58696047 -0.65282766]
y = [ 0.73020599 -0.73823951 -0.50705019  0.31286012]
The relative time delays computed with multi-plane lensing are:
dt_days = [ 8.01818853 14.50376897 18.01811095]
and the ones from a single plane with LOS corrections are:
dt_days = [ 8.01818853 14.50376897 18.01811095]

The difference is:
delta dt_days = [ 4.82884843e-11 -9.82751658e-11  5.80122617e-11]



The agreement is excellent

## 5. Test of LOS_Minimal

The line-of-sight model 'LOS_MINIMAL' inherits from 'LOS' with the following correspondance:
\begin{align}
\boldsymbol{\Gamma}_{\rm LOS} &= \boldsymbol{\Gamma}_{os}
\\
\boldsymbol{\Gamma}_{od} &= \boldsymbol{\Gamma}_{ds}
\end{align}
Here we test whether 'LOS_MINIMAL' has the expected behaviour.

In [31]:
# Define the two lens models, one with LOS_MINIMAL and one with LOS such that
# Gamma_ds = Gamma_od

kwargs_epl = {'theta_E': 0.8,
               'gamma': 1.95,
               'center_x': 0,
               'center_y': 0,
               'e1': .07,
               'e2': -0.03}

kappa_od = -0.05
omega_od = 0.02
gamma_od = np.array([0.05, 0])
kappa_los = 0.1
omega_los = 0.03
gamma_los = np.array([0.02, 0.1])
kappa_os, omega_os, gamma_os = kappa_los, omega_los, gamma_los
kappa_ds, omega_ds, gamma_ds = kappa_od, omega_od, gamma_od

kwargs_los = {
    'kappa_od': kappa_od, 'kappa_os': kappa_os, 'kappa_ds': kappa_ds,
    'omega_od': omega_od, 'omega_os': omega_os, 'omega_ds': omega_ds,
    'gamma1_od': gamma_od[0], 'gamma2_od': gamma_od[1],
    'gamma1_os': gamma_os[0], 'gamma2_os': gamma_os[1],
    'gamma1_ds': gamma_ds[0], 'gamma2_ds': gamma_ds[1]}

kwargs_minimal = {
    'kappa_od': kappa_od, 'kappa_los': kappa_los,
    'omega_od': omega_od, 'omega_los': omega_los,
    'gamma1_od': gamma_od[0], 'gamma2_od': gamma_od[1],
    'gamma1_los': gamma_os[0], 'gamma2_los': gamma_os[1]}

kwargs_lens_los = [kwargs_epl, kwargs_los]
kwargs_lens_minimal = [kwargs_epl, kwargs_minimal]

from lenstronomy.LensModel.lens_model import LensModel
lens_model_class_los = LensModel(['EPL', 'LOS'])
lens_model_class_minimal = LensModel(['EPL', 'LOS_MINIMAL'],
    z_source=z_s)

Adding LOS to the main lens.
Adding LOS_MINIMAL to the main lens.


### Testing ray shooting, Hessian and Fermat potential

In [32]:
# List of positions in the image plane
x, y = np.array([3,4,5]), np.array([2,1,0])

# Apply ray shooting to get the associated sources
beta_x_los, beta_y_los = lens_model_class_los.ray_shooting(x, y, kwargs_lens_los)
beta_x_minimal, beta_y_minimal = lens_model_class_minimal.ray_shooting(x, y, kwargs_lens_minimal)

# Hessian matrices
H_xx_los, H_xy_los, H_yx_los, H_yy_los = lens_model_class_los.hessian(x, y, kwargs_lens_los)
H_xx_minimal, H_xy_minimal, H_yx_minimal, H_yy_minimal = lens_model_class_minimal.hessian(x, y, kwargs_lens_minimal)

#time delays
fermat_pot_los = lens_model_class_los.fermat_potential(x, y, kwargs_lens_los)
fermat_pot_minimal = lens_model_class_minimal.fermat_potential(x, y, kwargs_lens_minimal)

print("""
For a list of images with coordinates:
x = {}
y = {}

The sources computed with 'LOS' are:
beta_x = {}
beta_y = {}
and the ones from a 'LOS_MINIMAL' are:
beta_x = {}
beta_y = {}
The difference is:
delta beta_x = {}
delta beta_y = {}

The Hessian matrix computed with 'LOS' is
H_xx = {}
H_xy = {}
H_yx = {}
H_yy = {}
while the Hessian matrix computed with 'LOS_MINIMAL is':
H_xx = {}
H_xy = {}
H_yx = {}
H_yy = {}
The difference is:
delta H_xx = {}
delta H_xy = {}
delta H_yx = {}
delta H_yy = {}

The Fermat potentials computed with 'LOS' are:
T = {}
and the ones from a 'LOS_MINIMAL' are:
T = {}
The difference is:
delta T = {}



""".format(x, y, beta_x_los, beta_y_los, beta_x_minimal, beta_y_minimal,
           beta_x_los - beta_x_minimal, beta_y_los - beta_y_minimal,
           H_xx_los, H_xy_los, H_yx_los, H_yy_los,
           H_xx_minimal, H_xy_minimal, H_yx_minimal, H_yy_minimal,
           H_xx_los - H_xx_minimal, H_xy_los - H_xy_minimal,
           H_yx_los - H_yx_minimal, H_yy_los - H_yy_minimal, 
           fermat_pot_los, fermat_pot_minimal,
           fermat_pot_los - fermat_pot_minimal
))


For a list of images with coordinates:
x = [3 4 5]
y = [2 1 0]

The sources computed with 'LOS' are:
beta_x = [1.80846439 2.62770739 3.54705468]
beta_y = [ 0.87584129  0.14657496 -0.63299205]
and the ones from a 'LOS_MINIMAL' are:
beta_x = [1.80846439 2.62770739 3.54705468]
beta_y = [ 0.87584129  0.14657496 -0.63299205]
The difference is:
delta beta_x = [0. 0. 0.]
delta beta_y = [0. 0. 0.]

The Hessian matrix computed with 'LOS' is
H_xx = [0.20333813 0.14260435 0.12852945]
H_xy = [-0.03771881  0.02069722  0.07845606]
H_yx = [0.01198476 0.07070373 0.12982992]
H_yy = [0.27137682 0.32985634 0.30775012]
while the Hessian matrix computed with 'LOS_MINIMAL is':
H_xx = [0.20333813 0.14260435 0.12852945]
H_xy = [-0.03771881  0.02069722  0.07845606]
H_yx = [0.01198476 0.07070373 0.12982992]
H_yy = [0.27137682 0.32985634 0.30775012]
The difference is:
delta H_xx = [0. 0. 0.]
delta H_xy = [0. 0. 0.]
delta H_yx = [0. 0. 0.]
delta H_yy = [0. 0. 0.]

The Fermat potentials computed with 'LOS' are:
T